# OpenAI Agent with LlamaIndex

## Install Dependencies

In [1]:
! pip install -qU llama-index==0.11.6 llama-index-llms-openai llama-index-readers-file llama-index-embeddings-openai "openinference-instrumentation-llama-index>=2" arize-phoenix python-dotenv

## Setup API Keys
To run the rest of the notebook you will need access to an OctoAI API key. You can sign up for an account [here](https://octoai.cloud/). If you need further guidance you can check OctoAI's [documentation page](https://octo.ai/docs/getting-started/how-to-create-octoai-access-token).

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_API_KEY"] = "key"

## Import libraries and setup LlamaIndex

In [3]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI


# Create an llm object to use for the QueryEngine and the ReActAgent
llm = OpenAI(model="gpt-4")

# Set up Phoenix

In [4]:
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor

In [5]:
import phoenix as px
session = px.launch_app()

/Users/jowb/Documents/ai_agents_cookbooks/agentenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [6]:
from phoenix.otel import register

tracer_provider = register()

LlamaIndexInstrumentor().instrument(tracer_provider=tracer_provider)

🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: default
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: localhost:4317
|  Transport: gRPC
|  Transport Headers: {'user-agent': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



## Load Documents

In [7]:
try:
    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/lyft"
    )
    lyft_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/uber"
    )
    uber_index = load_index_from_storage(storage_context)

    index_loaded = True
except:
    index_loaded = False

This is the point we create our vector indexes, by calculating the embedding vectors for each of the chunks. You only need to run this once.

In [13]:
if not index_loaded:
    # load data
    chicago_docs = SimpleDirectoryReader(
        input_files=["../city_data/Chicago.txt"]
    ).load_data()
    seattle_docs = SimpleDirectoryReader(
        input_files=["../city_data/Seattle.txt"]
    ).load_data()

    # build index
    chicago_index = VectorStoreIndex.from_documents(chicago_docs, show_progress=True)
    seattle_index = VectorStoreIndex.from_documents(seattle_docs, show_progress=True)

    # persist index
    # lyft_index.storage_context.persist(persist_dir="./storage/lyft")
    # uber_index.storage_context.persist(persist_dir="./storage/uber")

Generating embeddings: 100%|██████████| 18/18 [00:00<00:00, 33.74it/s]


Now create the query engines.

In [14]:
chicago_engine = chicago_index.as_query_engine(similarity_top_k=3, llm=llm)
seattle_engine = seattle_index.as_query_engine(similarity_top_k=3, llm=llm)

We can now define the query engines as tools that will be used by the agent.

As there is a query engine per document we need to also define one tool for each of them.

In [17]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=chicago_engine,
        metadata=ToolMetadata(
            name="chicago_wiki",
            description=(
                "Chicago's wiki"
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=seattle_engine,
        metadata=ToolMetadata(
            name="seattle_wiki",
            description=(
                "Seattle_wiki "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

## Creating the Agent
Now we have all the elements to create a LlamaIndex ReactAgent

In [18]:
agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
    max_turns=10,
)

Now we can interact with the agent and ask a question.

In [24]:
# response = agent.chat("Who had more profit in 2021, Lyft or Uber?")
response = agent.chat("which city has higher education in general?")
print(str(response))

> Running step 6a4421dc-731f-43da-b5b0-3efc56a3b8c0. Step input: which city has higher education in general?
Thought: The user is asking about the general level of education in Chicago and Seattle. I can use the chicago_wiki and seattle_wiki tools to gather information about the education levels in these cities.
Action: chicago_wiki
Action Input: {'input': 'education in Chicago'}
Observation: Chicago's education system is extensive and diverse. The Chicago Public Schools (CPS) governs over 600 public elementary and high schools, including several selective-admission magnet schools. There are eleven selective enrollment high schools designed to meet the needs of the city's most academically advanced students. The district is the third-largest in the U.S., with an enrollment exceeding 400,545 students. 

Chicago also has a network of Lutheran schools, private schools run by various denominations and faiths, and completely secular private schools. There are also specialized arts-focused s